In [ ]:
from torchvision import models
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class MyBasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, stride=1)
        self.BN1 = nn.BatchNorm2d(out_channels)
        self.BN2 = nn.BatchNorm2d(out_channels)
        if in_channels != out_channels or stride != 1:
            self.skip = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
            nn.BatchNorm2d(out_channels))

        else:
            self.skip = nn.Identity()

    def forward(self, x):
        out = F.relu(self.BN1(self.conv1(x)))
        out = self.BN2(self.conv2(out))
        skip_x = self.skip(x)
        out = out + skip_x
        out = F.relu(out)
        return out

In [ ]:
class MyResNet(nn.Module):
    def __init__(self):
      super().__init__()
      self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
      self.layer1 = nn.Sequential(MyBasicBlock(64, 64), MyBasicBlock(64, 64))
      self.layer2 = nn.Sequential(MyBasicBlock(64, 128, stride=2), MyBasicBlock(128, 128))
      self.layer3 = nn.Sequential(MyBasicBlock(128, 256, stride=2), MyBasicBlock(256, 256))
      self.layer4 = nn.Sequential(MyBasicBlock(256, 512, stride=2), MyBasicBlock(512, 512))
      self.BN1 = nn.BatchNorm2d(64)
      self.relu = nn.ReLU(inplace=True)

      self.pool = nn.AdaptiveAvgPool2d(1)
      self.fc = nn.Linear(in_features=512, out_features=10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.BN1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

100%|██████████| 170M/170M [00:03<00:00, 48.1MB/s]


In [ ]:
model = MyResNet()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.9)

In [ ]:
x = next(iter(train_loader))[0]
print(x.shape)  # [128, 3, 32, 32]

model.eval()
with torch.no_grad():
    out = model(x.to(device))
    print(out.shape)  # [128, 10]

torch.Size([128, 3, 32, 32])
torch.Size([128, 10])


In [ ]:
def evaluate(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted==labels).sum().item()
    return 100 * correct / total

In [ ]:
def train(model, train_loader, test_loader, optimizer, criterion, device, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_loss = running_loss / len(train_loader)
        acc = evaluate(model, test_loader, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {acc:.2f}%")

In [ ]:
train(model, train_loader, test_loader, optimizer, criterion, device, num_epochs=5)

Epoch 1/5, Loss: 1.2774, Accuracy: 60.22%
Epoch 2/5, Loss: 0.7383, Accuracy: 70.33%
Epoch 3/5, Loss: 0.5116, Accuracy: 76.72%
Epoch 4/5, Loss: 0.3703, Accuracy: 76.48%
Epoch 5/5, Loss: 0.2399, Accuracy: 78.76%
